In [3]:
import requests
import json
from tabulate import tabulate
# import coki.funcs as funcs
import funcs
import pandas as pd
from datetime import datetime
import os
import importlib
import numpy as np

In [23]:
importlib.reload(funcs)

<module 'funcs' from 'c:\\Lucas\\Coding\\Github\\cocos-precios\\src\\coki\\funcs.py'>

In [4]:
# link = 'https://api.cocos.capital/api/v1/wallet/portfolio'
token = funcs.get_access_token()
account_id = funcs.get_account_id(token)

In [1]:

def get_portfolio(op_type="all"):
    link = f"https://api.cocos.capital/api/v1/wallet/portfolio"

    token = funcs.token_promise()
    account_id = funcs.account_promise(token)

    j = funcs.request_info(link, token, account_id, info_type="wallet")

    time_str = datetime.now().strftime("%H:%M:%S")

    mail = funcs.read_credentials()[0]

    # disponible = {'Total en Pesos': j['total']['ars'], 'Total en Dolares': j['total']['usd']}
    portfolio = j["tickers"]

    tables = []
    for i in range(0, len(portfolio)):
        data = portfolio[i]

        table = {
            "Ticker": data["ticker"],
            "Cantidad": data["quantity"],
        }
        tables.append(table)
    df = pd.DataFrame(tables)
    df = df.copy().set_index("Ticker")

    if op_type == "all":
        print(
            "\n",
            time_str,
            "\n",
            mail,
            "\n",
            # f'Porfolio total en ARS: $ {int(j["total"]["ars"]):,}',
            # "\n",
            # f'Porfolio total en USD: $ {int(j["total"]["usd"]):,}',
        )
        print(
            tabulate(
                df,
                headers="keys",
                tablefmt="rounded_outline",
                floatfmt=".0f",
                stralign="center",
            )
        )
        d = df.iloc[:2].to_dict("index")
        return (d["AR$"]["Cantidad"], d["US$ MEP"]["Cantidad"])
    
    if op_type == "buy":
        print(
            "\n",
            time_str,
            "\n",
            mail,
            "\n",
            # f'Porfolio total en ARS: $ {int(j["total"]["ars"]):,}',
            # "\n",
            # f'Porfolio total en USD: $ {int(j["total"]["usd"]):,}',
        )
        print(
            tabulate(
                df.iloc[:2],
                headers="keys",
                tablefmt="rounded_outline",
                floatfmt=".0f",
                stralign="center",
            )
        )
        d = df.iloc[:2].to_dict("index")
        return (d["AR$"]["Cantidad"], d["US$ MEP"]["Cantidad"])
    
    if op_type == "sell":
        print(
            "\n",
            time_str,
            "\n",
            mail,
            "\n",
            # f'Porfolio total en ARS: $ {int(j["total"]["ars"]):,}',
            # "\n",
            # f'Porfolio total en USD: $ {int(j["total"]["usd"]):,}',
        )
        print(
            tabulate(
                df.iloc[2:],
                headers="keys",
                tablefmt="rounded_outline",
                floatfmt=".0f",
                stralign="center",
            )
        )
        d = df.iloc[2:].to_dict("index")
        return d    

In [34]:
def calcula_op_sell(
    cantidad: str, disponible: dict, precio: float, currency: str, data: dict, ticker: str
):
    """
    Calcula el numero de acciones a vender
    """
    if (ticker[-1] in ["D", "C"]) and currency == 'USD':
            ticker = ticker[:-1]
    if cantidad == "a":
        if currency == "USD":
            acciones = int(disponible[ticker]['Cantidad'])
        elif currency == "ARS":
            acciones = int(disponible[ticker]['Cantidad'])
    elif cantidad[-1] == "a":
        acciones = int(cantidad[:-1])
    elif cantidad[-1] == "%":
        acciones = int(cantidad[:-1]) / 100 * int(disponible[ticker]['Cantidad'])
    # elif cantidad[-1] not in ["a", "%"]:
    #     acciones = int(int(cantidad) / precio)

    assert acciones > 0, "No se puede operar con esa cantidad"
    # if currency == "USD":
    #     assert (
    #         acciones * precio < disponible[1]
    #     ), f"No hay suficiente dinero para operar - Disponible: {round(disponible[1],2)}"
    # elif currency == "ARS":
    #     assert (
    #         acciones * precio < disponible[0]
    #     ), f"No hay suficiente dinero para operar - Disponible: {round(disponible[0],2)}"

    tick_size = data["tick_size"]
    assert (
        precio / tick_size
    ).is_integer(), f'El precio no es multiplo de ({data["tick_size"]})'

    if precio > data["last"] * 1.05:
        correct = input(
            f'El precio elegido es un 5% mayor al último operado ({data["last"]}). ¿Es correcto? (S/N): '
        )
        while correct not in ["S", "N", "s", "n"]:
            correct = input(
                f'El precio elegido es un 5% mayor al último operado ({data["last"]}). ¿Es correcto? (S/N): '
            )
        if correct in ["N", "n"]:
            raise ValueError("Precio incorrecto")

    elif precio < data["last"] * 0.95:
        correct = input(
            f'El precio elegido es más que 5% menor al último operado ({data["last"]}). ¿Es correcto? (S/N): '
        )
        while correct not in ["S", "N", "s", "n"]:
            correct = input(
                f'El precio elegido es más que 5% menor al último operado ({data["last"]}). ¿Es correcto? (S/N): '
            )
        if correct in ["N", "n"]:
            raise ValueError("Precio incorrecto")

    return acciones


In [27]:
ticker = 'KOD'

In [28]:
disponible[ticker[:-1]]

{'Cantidad': 25.0}

In [22]:
disponible.keys()

dict_keys(['GOOGL', 'KO', 'MA', 'WMT', 'EWZ'])

In [19]:
disponible = get_portfolio(op_type="sell")


 15:54:57 
 nicolaspcastro@gmail.com 

╭──────────┬────────────╮
│  Ticker  │   Cantidad │
├──────────┼────────────┤
│  GOOGL   │        171 │
│    KO    │         25 │
│    MA    │         20 │
│   WMT    │          5 │
│   EWZ    │          3 │
╰──────────┴────────────╯


In [35]:
calcula_op_sell(cantidad='50%', disponible=disponible, precio=11.50, currency='USD', data=data, ticker='KOD')

12.5

In [82]:
account_id

55743

In [59]:
funcs.get_portfolio(op_type = 'sell')


 16:44:39 
 nicolaspcastro@gmail.com 

╭──────────┬────────────╮
│  Ticker  │   Cantidad │
├──────────┼────────────┤
│    KO    │         25 │
│    MA    │         20 │
│   WMT    │          5 │
│  GOOGL   │         35 │
│   EWZ    │          3 │
╰──────────┴────────────╯


In [5]:
disponible = funcs.get_portfolio(op_type = 'buy')


 16:58:53 
 nicolaspcastro@gmail.com 

╭──────────┬────────────╮
│  Ticker  │   Cantidad │
├──────────┼────────────┤
│   AR$    │       4587 │
│ US$ MEP  │         25 │
╰──────────┴────────────╯


In [29]:
np.remainder(11.50,0.01)

0.00999999999999976

In [31]:
11.50/0.01

1150.0

In [37]:
11.5%0.05

0.049999999999999364

In [33]:
np.isclose(2.31%0.01, 0, atol=0.01)

True

In [36]:
data

{'short_ticker': 'KOD',
 'long_ticker': 'KOD-0003-C-CT-USD',
 'instrument_code': 'KO',
 'ext_code_cv': 8006,
 'instrument_name': 'Cedear Coca Cola Company',
 'instrument_short_name': 'Coca-Cola',
 'instrument_type': 'CEDEARS',
 'instrument_subtype': 'OTROS',
 'maturity': None,
 'expires_at': None,
 'logo_file_name': 'KO.jpg',
 'id_venue': 'BYMA',
 'id_session': 'CT',
 'id_segment': 'C',
 'settlement_days': 2,
 'currency': 'USD',
 'price_factor': 1,
 'contract_size': 1,
 'min_lot_size': 1,
 'id_security': 15468,
 'tick_size': 0.05,
 'date': '2023-12-20',
 'open': 11.95,
 'high': 11.95,
 'low': 11.5,
 'close': 11.6,
 'prev_close': 11.7,
 'last': 11.6,
 'bid': 11.5,
 'ask': 11.7,
 'bids': [{'size': 90, 'price': 11.5},
  {'size': 86, 'price': 11.4},
  {'size': 17, 'price': 11.35},
  {'size': 5, 'price': 11.3},
  {'size': 9, 'price': 11.25}],
 'asks': [{'size': 500, 'price': 11.7},
  {'size': 961, 'price': 11.75},
  {'size': 483, 'price': 11.8},
  {'size': 1068, 'price': 11.85},
  {'size': 

In [41]:
precio = 11.50
cantidad = '100'
# disponible = (90,80)
data, ticker_link = funcs.gen_table('KOD')

	 KOD - Cedear Coca Cola Company - USD - 17:36:14
	 Last: 11.6 - High: 11.95 - Low: 11.5 - Volume: 45360
╭────────────────┬─────────────────┬────┬────────────────┬───────────────╮
│   Cant. compra │   Precio compra │    │   Precio venta │   Cant. venta │
├────────────────┼─────────────────┼────┼────────────────┼───────────────┤
│              5 │           11.30 │    │          11.95 │           343 │
│             10 │           10.80 │    │          12.25 │            10 │
│             10 │            9.10 │    │          12.30 │           139 │
│                │                 │    │          12.40 │           396 │
│                │                 │    │          12.50 │             1 │
╰────────────────┴─────────────────┴────┴────────────────┴───────────────╯


In [38]:
def calcula_operaciones(cantidad: str, disponible: tuple, precio: float, currency: str, data: dict):
    """
    Calcula el numero de acciones a comprar o vender
    """

    if cantidad == 'a':
        if currency == 'USD':
            acciones = int(disponible[1]/precio)
        elif currency == 'ARS':
            acciones = int(disponible[0]/precio)
    elif cantidad[-1] == 'a':
        acciones = int(cantidad[:-1])
    elif cantidad[-1] == '%':
        if currency == 'USD':
            acciones = int(disponible[1]*int(cantidad[:-1])/100/precio)
        elif currency == 'ARS':
            acciones = int(disponible[0]*int(cantidad[:-1])/100/precio)
    elif cantidad[-1] not in ['a', '%']:
        acciones = int(int(cantidad)/precio)

    assert acciones > 0, 'No se puede operar con esa cantidad'
    if currency == 'USD':
        assert acciones*precio > disponible[1], f'No hay suficiente dinero para operar - Disponible: {disponible[1]}'
    elif currency == 'ARS':
        assert acciones*precio > disponible[0], f'No hay suficiente dinero para operar - Disponible: {disponible[0]}'
    
    tick_size = data['tick_size']
    assert precio/tick_size%10 == 0, f'El precio no es multiplo de ({data["tick_size"]})'
    
    if precio > data['last']*1.05: 
        correct = input(f'El precio elegido es un 5% mayor al último operado ({data["last"]}). ¿Es correcto? (S/N): ')
        while correct not in ['S', 'N', 's', 'n']:
            correct = input(f'El precio elegido es un 5% mayor al último operado ({data["last"]}). ¿Es correcto? (S/N): ')
        if correct in ['N', 'n']:
            raise ValueError('Precio incorrecto')
        
    elif precio < data['last']*0.95:
        correct = input(f'El precio elegido es un 5% menor al último operado ({data["last"]}). ¿Es correcto? (S/N): ')
        while correct not in ['S', 'N', 's', 'n']:
            correct = input(f'El precio elegido es un 5% menor al último operado ({data["last"]}). ¿Es correcto? (S/N): ')
        if correct in ['N', 'n']:
            raise ValueError('Precio incorrecto')
            
    return acciones


In [39]:
currency = 'USD'

In [40]:
# pasar a función
calcula_operaciones(cantidad, disponible, precio, currency, data)

8